# Capstone project week 3. IBM Data Science
## Segmenting and Clustering Neighborhoods in Toronto

#### Import libraries

In [350]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # Library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries loaded.')

Libraries loaded.


### This notebook contains all 3 part of week 3 project

# Week 3 part 1

### Scrape Wikipedia page to obtain data and transform into panda datafram using BeatifulSoup

In [351]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #wikipedia link to the table
headers = {'User-Agent': 'Mozilla/5.0'} #setting my browser

In [352]:
#parse data from the html with beautifulsoup

response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.content, "html.parser")

In [353]:
# creating empty list for the three in the table to store data
postCodeList = []
boroughList = []
neighborList = []

In [354]:
## accessing the table and saving it under vairable
wikitable = soup.find('table', class_="wikitable")

## loop through the table cell by cell and appending content to corrosponding column

In [355]:
# iterating through the table to get the contents
for row in wikitable.find_all("tr"): #finding row in the table
    cell = row.find_all("td") #finding cells in the row
    if(len(cell) > 0):
        #appending list to corresponding column
        postCodeList.append(cell[0].text)
        boroughList.append(cell[1].text)
        neighborList.append(cell[2].text.rstrip('\n'))

### Transforming the data in to pandas DataFram

In [356]:
#creating columns for corrosponding lists
columns ={"PostCode": postCodeList, "Borough": boroughList, "Neighborhood": neighborList}
wiki_df = pd.DataFrame(columns)
wiki_df.head(12)

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


## Cleaning data: drop cells with borough not assigned

In [357]:
# drop cells with borough not assigned
wiki_df_dropna = wiki_df[wiki_df.Borough != "Not assigned"].reset_index(drop=True)
wiki_df_dropna.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [358]:
# group neighborhoods with same postal code area
wiki_df_grouped = wiki_df_dropna.groupby(["PostCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
wiki_df_grouped.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [359]:
# for a borough with no  neighborhood assigned, set borough as neighborhood 
for index, row in wiki_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
wiki_df_grouped.head(10).set_index("PostCode", inplace=True)

## Printing the shape of the datafram

In [360]:
wiki_df.shape
wiki_df_grouped.shape

(103, 3)

# Week 3 part 2

##  Load the coordinates from the csv file

In [361]:
# The code was removed by Watson Studio for sharing.

In [362]:
# Load the coordinates from the csv file
coordinates = pd.read_csv(Geospatial_Coordinates)

# renaming postal code column
coordinates.rename(columns={"Postal Code": "PostCode"}, inplace=True)
coordinates.head()


,PostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the table and coordinates

In [363]:
#Merging the table and coordinates
merged_df = wiki_df_grouped.merge(coordinates, on="PostCode", how="left")
merged_df.head(12)

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [364]:
# column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
# last = pd.DataFrame(columns=column_names)

# test_list = coordinates["PostCode"]

# for postcode in test_list:
#     test_df = last.append(merged_df[merged_df["PostCode"]==postcode], ignore_index=True)
    
# last

# Week 3 part 3


## Exploring and cluster the neighborhoods in Toronto

## Using geopy to define a geocoder instance

In [365]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate for city of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate for city of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods with folium

In [366]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#07ded2',
        fill=True,
        fill_color='#eb5a46',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Segment and cluster neighborhoods North York only

In [367]:
North_York = merged_df[merged_df['Borough'] == 'North York'].reset_index(drop=True)
North_York

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


## geographical coordinates of North York

In [368]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [369]:
## Creating map for North York using the corrosponding latitude and longitude values

In [370]:
# create map of North York with latitude & longitude values
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(North_York['Latitude'], North_York['Longitude'], North_York['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North_York)  
    
map_North_York

## Using Foursquare API to explore North York neighborhood

In [371]:
# The code was removed by Watson Studio for sharing.

In [372]:
# Getting name of neighborhood at index 17
North_York.loc[17, 'Neighborhood']

'Victoria Village'

In [373]:
# get geo coordinate for Willowdale south
neighborhood_latitude = North_York.loc[17, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_York.loc[17, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_York.loc[17, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


# Explore top 50 venues in Victoria Village with in 500 meters

In [374]:
# Explore top 50 venues in Victoria Village with in 500 meters
radius = 500
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


# Send Get request


In [375]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dae8579446ea600391c8014'},
 'response': {'headerLocation': 'Bermondsey',
  'headerFullLocation': 'Bermondsey, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7303823045, 'lng': -79.30935618239715},
   'sw': {'lat': 43.72138229549999, 'lng': -79.32178701760282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c633acb86b6be9a61268e34',
       'name': 'Victoria Village Arena',
       'location': {'lat': 43.72348055545508,
        'lng': -79.31563520925143,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -79.31563520925143}],
        'distance': 267,
        'cc': 'CA',
        'country': 'Canada',
        'formatte

# Extracting venue category  from Foursquare lab

In [376]:
# function to get venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Transforming json into a pandas dataframe

In [377]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Tim Hortons,Coffee Shop,43.725517,-79.313103
2,Portugril,Portuguese Restaurant,43.725819,-79.312785
3,Pizza Nova,Pizza Place,43.725824,-79.312860


# Size of North York dataframe

In [378]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


# Explore Neighborhoods in North York

In [379]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Creating dataframe for venues called North_York_venues

In [380]:
North_York_venues = getNearbyVenues(names=North_York['Neighborhood'],
                                   latitudes=North_York['Latitude'],
                                   longitudes=North_York['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [381]:
# checking the shape and printing sample
print(North_York_venues.shape)
North_York_venues.head()

(225, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


# Grouping venues for each nighborhood

In [382]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
"CFB Toronto, Downsview East",2,2,2,2,2,2
Don Mills North,4,4,4,4,4,4
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,7,7,7,7,7,7
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


# Analyzing Each Neighborhood

In [383]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [384]:
North_York_onehot.shape

(225, 102)

In [385]:
York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
York_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.00,0.000000,0.055556,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.055556,0.0,0.00,0.000000,0.055556,0.055556,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.055556,0.000000,0.000000,0.0,0.000000,0.000000,0.055556,0.055556,0.000000,0.0,0.00,0.000000,0.000000,0.055556,0.00,0.055556,0.00,0.055556,0.00,0.000000,0.000000,0.000000,0.055556,0.00,0.055556,0.00,0.000000,0.00,0.00,0.00,0.055556,0.000000,0.00
1,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.250000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.043478,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.00,0.00,0.043478,0.043478,0.00,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.043478,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.043478,0.043478,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.043478,0.000000,0.086957,0.000000,0.00,0.086957,0.043478,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.043478,0.043478,0.000000,0.0,0.00,0.043478,0.000000,0.043478,0.00,0.043478,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.043478,0.00,0.043478,0.00,0.00,0.00,0.000000,0.000000,0.00
3,"CFB Toronto, Downsview East",0.000000,0.5,0.000000,0.000000,0

# Cluster Neighborhoods

In [386]:
# set number of clusters
kclusters = 3

York_grouped_clustering = York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 1, 2, 1, 1, 1, 2], dtype=int32)

In [387]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [388]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = York_grouped['Neighborhood']

for ind in np.arange(York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(York_grouped.iloc[ind, :], top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Bank,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Wings Joint,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Juice Bar,Coffee Shop,Grocery Store,Sandwich Place,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Liquor Store,Comfort Food Restaurant
3,"CFB Toronto, Downsview East",Airport,Park,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Wings Joint,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


# create new dataframe cluster each neighborhood

In [389]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

York_merged = North_York

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
York_merged = York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

York_merged.head() # check the last columns!

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Dog Run,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Tea Room,Electronics Store,Department Store,Deli / Bodega,Cosmetics Shop,Food Court,Restaurant
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Wings Joint,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0.0,Park,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Visualize the clusters results

In [396]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(York_merged['Latitude'], York_merged['Longitude'], York_merged['PostCode'], York_merged['Borough'], York_merged['Neighborhood'], York_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

## Cluster 1

In [398]:
York_merged.loc[York_merged['Cluster Labels'] == 0, York_merged.columns[[1] + list(range(3, York_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,43.757490,-79.374714,0.0,Park,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
8,North York,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
13,North York,43.737473,-79.464763,0.0,Airport,Park,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 2

In [399]:
York_merged.loc[York_merged['Cluster Labels'] == 1, York_merged.columns[[1] + list(range(3, York_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,43.803762,-79.363452,1.0,Dog Run,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,North York,43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Tea Room,Electronics Store,Department Store,Deli / Bodega,Cosmetics Shop,Food Court,Restaurant
2,North York,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Wings Joint,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
5,North York,43.770120,-79.408493,1.0,Ramen Restaurant,Coffee Shop,Restaurant,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Grocery Store,Liquor Store,Pet Store
6,North York,43.752758,-79.400049,1.0,Park,Convenience Store,Bank,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
7,North York,43.782736,-79.442259,1.0,Grocery Store,Butcher,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Athletics & Sports,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
9,North York,43.745906,-79.352188,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Wings Joint,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
10,North York,43.725900,-79.340923,1.0,Coffee Shop,Beer Store,Asian Restaurant,Gym,Clothing Store,Chinese Restaurant,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant,Restaurant
11,North York,43.754328,-79.442259,1.0,Coffee Shop,Fried Chicken Joint,Bank,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place
12,North York,43.767980,-79.487262,1.0,Coffee Shop,Furniture / Home Store,Caribbean Restaurant,Bar,Falafel Restaurant,Massage Studio,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store


## Cluster 3

In [400]:
York_merged.loc[York_merged['Cluster Labels'] == 2, York_merged.columns[[1] + list(range(3, York_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,43.728496,-79.495697,2.0,Home Service,Food Truck,Baseball Field,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
23,North York,43.724766,-79.532242,2.0,Paper / Office Supplies Store,Baseball Field,Wings Joint,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
